# Basic Tutorial: Getting Started with t-prompts

This tutorial introduces the core concepts of `t-prompts`: creating structured prompts using Python 3.14's t-strings that preserve provenance while rendering to plain text.

## What are t-strings?

Python 3.14 introduced **template strings** (t-strings), which look like f-strings but preserve information about interpolations instead of immediately evaluating to a string.

Unlike f-strings that give you `"Hello Alice"`, t-strings give you a `Template` object that remembers the expression `name` and its value `"Alice"`.

In [1]:
# Compare f-strings vs t-strings
name = "Alice"

# f-string: immediately evaluates to a string
f_result = f"Hello {name}"
print(f"f-string type: {type(f_result)}")
print(f"f-string value: {f_result}")

# t-string: returns a Template object with metadata
t_result = t"Hello {name}"
print(f"\nt-string type: {type(t_result)}")
print(f"t-string: {t_result}")
print(f"Expression preserved: {t_result.interpolations[0].expression}")

f-string type: <class 'str'>
f-string value: Hello Alice

t-string type: <class 'string.templatelib.Template'>
t-string: Template(strings=('Hello ', ''), interpolations=(Interpolation('Alice', 'name', None, ''),))
Expression preserved: name


## Creating Your First Structured Prompt

The `prompt()` function wraps a t-string into a `StructuredPrompt` that acts like both a string and a navigable tree.

In [2]:
from t_prompts import prompt

instructions = "Always answer politely."
p = prompt(t"Obey {instructions:inst}")

# Renders like an f-string
print(f"Rendered: {str(p)}")

# But preserves provenance
node = p['inst']
print(f"\nExpression: {node.expression}")
print(f"Value: {node.value}")

Rendered: Obey Always answer politely.

Expression: instructions
Value: Always answer politely.


## Format Specs as Keys

The format spec (the part after `:`) becomes the **key** for accessing interpolations. Without a format spec, the expression itself is used as the key.

In [3]:
# No format spec: key is the expression
user_query = "What is Python?"
p1 = prompt(t"Question: {user_query}")
print(f"No format spec - Key: {list(p1.keys())[0]}")
print(f"Value: {p1['user_query'].value}")

# With format spec: key is the format spec
p2 = prompt(t"Question: {user_query:query}")
print(f"\nWith format spec - Key: {list(p2.keys())[0]}")
print(f"Value: {p2['query'].value}")

No format spec - Key: user_query
Value: What is Python?

With format spec - Key: query
Value: What is Python?


## Accessing Interpolations

You can access interpolations like a dictionary and iterate over them.

In [4]:
context = "User is Alice"
task = "Translate to French"
p = prompt(t"Context: {context:ctx}. Task: {task:t}")

# Check if key exists
print(f"Has 'ctx': {'ctx' in p}")

# Get all keys
print(f"Keys: {list(p.keys())}")

# Iterate over interpolations
for key, node in p.items():
    print(f"{key}: {node.value}")

Has 'ctx': True
Keys: ['ctx', 't']
ctx: User is Alice
t: Translate to French


## Rendering to Text

The `str()` function converts a `StructuredPrompt` to text, just like an f-string would.

In [5]:
name = "Bob"
greeting = "Hello"
p = prompt(t"{greeting:g}, {name:n}!")

# Convert to string
text = str(p)
print(f"Rendered text: {text}")
print(f"Type: {type(text)}")

Rendered text: Hello, Bob!
Type: <class 'str'>


## Type Safety

Only strings and nested `StructuredPrompt` objects are allowed as values. This prevents accidental `str(obj)` conversions.

In [6]:
# This works: string values
msg = "Hello"
p1 = prompt(t"{msg:message}")
print(f"String value: {str(p1)}")

# This works: nested StructuredPrompt
p2 = prompt(t"{p1:nested}")
print(f"Nested prompt: {str(p2)}")

# This fails: non-string, non-prompt objects
try:
    number = 42
    p3 = prompt(t"{number:num}")
except Exception as e:
    print(f"\nError (expected): {type(e).__name__}")
    print(f"Message: {str(e)[:80]}...")

String value: Hello
Nested prompt: Hello

Error (expected): UnsupportedValueTypeError
Message: Unsupported value type for interpolation 'number' (key='num'): expected str or S...


## Next Steps

You've learned the basics! Continue to:

- **02-intermediate.ipynb**: Learn about nested prompts, navigation, and conversions
- **03-ir-visualization.ipynb**: See the widget visualization in action
- **topics/**: Deep dives into specific features (few-shot prompts, source mapping, dedenting, etc.)